In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from  scipy.stats import norm,skew
import time

In [72]:
train=pd.read_csv("data/train.csv", parse_dates=["datetime"])
test=pd.read_csv("data/test.csv", parse_dates=["datetime"])
train["cnt"]=train["count"]
train.drop(["count"],axis=1,inplace=True)

In [73]:
n_train=train.shape[0]
n_test=test.shape[0]
y_train=train.cnt.values
test_id=test.datetime
print (n_train,n_test)

10886 6493


In [74]:
target_col=["casual","registered","cnt"]
for col in target_col:
    train["%s_log"%col]=np.log(train[col]+1)

In [75]:
for df in [train,test]:
    df["year"],df["month"],df["hour"],df["dayofweek"]=\
        df.datetime.dt.year,df.datetime.dt.month,\
        df.datetime.dt.hour,df.datetime.dt.dayofweek

In [76]:
all_data= [train,test]

In [77]:
for df in all_data:
    df["year_season"]=df["year"]+df["season"]/10

In [78]:
for df in all_data:
    df["hour_workingday_casual"]=df[["hour"]].apply(
        lambda x : int (10<=x["hour"]<=19),axis=1)

In [79]:
for df in all_data:
    df["hour_working_registered"]=df[["hour","workingday"]].apply(
    lambda x : int (
                (x["workingday"]==1 and (x["hour"]==8 or 17<=x["hour"]<=18))
                or (x["workingday"]==0 and 10<=x["hour"]<=19)), axis=1)

In [80]:
by_season=train.groupby("year_season")[["cnt"]].median()

In [81]:
by_season.columns=["count_season"]

In [82]:
train=train.join(by_season,on="year_season")
test=test.join(by_season,on="year_season")

In [87]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

c:\python\python36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [93]:
def rmsle(y, y_pred):
    return np.sqrt(np.mean(np.square(np.log(y + 1) - np.log(y_pred + 1))))

In [94]:
preds={}
regs={"gbt":GradientBoostingRegressor(random_state=10),
     "rf":RandomForestRegressor(random_state=0,n_jobs=-1)}

In [97]:
for name,reg in regs.items():
    features=["season","holiday","workingday","weather",
             "temp","atemp","humidity","windspeed","year","hour",
             "dayofweek","hour_workingday_casual","count_season"]
    if name=="gbt":
        reg.set_params(n_estimators=1000,min_samples_leaf=6)
    elif name=="rf":
        reg.set_params(n_estimators=1000,min_samples_leaf=2)
    reg.fit(train[features],train["casual_log"])    
    pred_casual=reg.predict(test[features])
    pred_casual=np.exp(pred_casual)-1
    pred_casual[pred_casual<0]=0
    
    train_pred_casual=reg.predict(train[features])
    train_pred_casual=np.exp(train_pred_casual)-1
    
    features=["season","holiday","workingday","weather",
             "temp","atemp","humidity","windspeed","year","hour",
             "dayofweek","hour_working_registered","count_season"]
    if name=="gbt":
        reg.set_params(n_estimators=1000,min_samples_leaf=6)
    elif name=="rf":
        reg.set_params(n_estimators=1000,min_samples_leaf=2)
    reg.fit(train[features],train["registered_log"])
    pred_registered=reg.predict(test[features])
    pred_registered=np.exp(pred_registered)-1
    pred_registered[pred_registered<0]=0
    preds[name]=pred_casual+pred_registered

    train_pred_registered=reg.predict(train[features])
    train_pred_registered=np.exp(train_pred_registered)-1
    print(name,rmsle(train_pred_casual*0.7+train_pred_registered*0.3, y_train))
    
    
pred=0.7*preds["gbt"]+0.3*preds["rf"]    

gbt 1.0138250367012605
rf 0.9892892189048377


In [98]:
sub = pd.DataFrame()
sub['datetime'] = test_id
sub['count'] = pred

from datetime import datetime
now=datetime.now()
sub.to_csv("data/{0:02d}{1:02d}{2:02d}{3:02d}_ensemble_submission.csv".format(now.year,now.month,now.day,now.hour),index=False)

In [99]:
24/3251

0.007382343894186404